In [2]:
# 멘티1명 멘토 10명 임시 정의
mentee = {
    "name": "최지원",
    "experience": "기초적인 Python 프로그래밍 경험이 있고, 머신러닝 입문 과정을 수강하였습니다. Kaggle에서 간단한 데이터 분석 프로젝트를 수행해보았습니다.",
    "goals": "머신러닝 및 데이터 분석 역량을 키워, 향후 AI 연구자로 성장하고 싶습니다.",
    "level": "초급"
}



mentors = [
    {
        "name": "김현우",
        "mentoring_area": ["데이터 분석", "Python", "통계"],
        "skills": ["Python", "Pandas", "Matplotlib", "Seaborn", "머신러닝"],
        "desired_mentee_level": "초급"
    },
    {
        "name": "박지훈",
        "mentoring_area": ["빅데이터", "데이터 엔지니어링", "SQL"],
        "skills": ["SQL", "Spark", "Hadoop", "ETL", "데이터 파이프라인"],
        "desired_mentee_level": "중급"
    },
    {
        "name": "이서연",
        "mentoring_area": ["AI 모델링", "딥러닝", "컴퓨터 비전"],
        "skills": ["TensorFlow", "Keras", "OpenCV", "PyTorch"],
        "desired_mentee_level": "초급"
    },
    {
        "name": "정민수",
        "mentoring_area": ["Java", "백엔드 개발", "API 설계"],
        "skills": ["Java", "Spring Boot", "MySQL", "REST API", "Maven"],
        "desired_mentee_level": "중급"
    },
    {
        "name": "최하영",
        "mentoring_area": ["마케팅", "데이터 분석", "SNS 운영"],
        "skills": ["Excel", "PowerPoint", "SEO", "SNS 광고"],
        "desired_mentee_level": "초급"
    },
    {
        "name": "한재민",
        "mentoring_area": ["데이터 시각화", "Python", "BI 도구"],
        "skills": ["Tableau", "Power BI", "Python", "시각화 기법"],
        "desired_mentee_level": "초급"
    },
    {
        "name": "이하늘",
        "mentoring_area": ["법률 상담", "지적 재산권"],
        "skills": ["지적재산권", "계약법", "민사소송법"],
        "desired_mentee_level": "상급"
    },
    {
        "name": "류진영",
        "mentoring_area": ["데이터 사이언스", "머신러닝", "AI"],
        "skills": ["Scikit-learn", "Python", "R", "머신러닝 알고리즘"],
        "desired_mentee_level": "초급"
    },
    {
        "name": "김나영",
        "mentoring_area": ["프론트엔드 개발", "UI/UX 디자인"],
        "skills": ["HTML", "CSS", "JavaScript", "React", "Figma"],
        "desired_mentee_level": "중급"
    },
    {
        "name": "오승민",
        "mentoring_area": ["모바일 앱 개발", "iOS 개발"],
        "skills": ["Swift", "Objective-C", "Xcode", "UI 설계"],
        "desired_mentee_level": "상급"
    }
]



In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\smhrd1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import re
from konlpy.tag import Okt
from sklearn.metrics.pairwise import cosine_similarity
from transformers import ElectraPreTrainedModel, ElectraModel,AutoTokenizer
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 한국어 불용어 목록 정의 (실제 불용어 목록을 여기에 넣으세요)
korean_stopwords = []  # 불용어 업슨ㄴ게 값이 더 잘나옴

# 영어 불용어 목록 정의
english_stopwords = nltk_stopwords.words('english')

# Komoran 형태소 분석기 인스턴스 생성
okt = Okt()

# KoELECTRA 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert", trust_remote_code=True)


# 맞춤형 모델 클래스 정의
class CustomElectraForSequenceClassification(ElectraPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.electra = ElectraModel(config)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None,
        output_hidden_states=True,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.electra(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]
        pooled_output = sequence_output[:, 0, :]  # [CLS] 토큰의 임베딩

        logits = self.classifier(pooled_output)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return output

        return {'logits': logits, 'hidden_states': outputs.hidden_states}

# 모델 로드
model = CustomElectraForSequenceClassification.from_pretrained('monologg/koelectra-base-v3-discriminator', num_labels=1, ignore_mismatched_sizes=True)
state_dict = torch.load('../models/KoELECTRA_base.pth', map_location=torch.device('cuda'))
model.load_state_dict(state_dict)
model.eval()

def preprocess_text(text):
    # 전처리
    text = re.sub(r'[^ㄱ-ㅎ가-힣a-zA-Z\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.lower()
    tokens = okt.pos(text)
    
    # 품사 선택에 맞게 수정
    selected_pos = ['Noun', 'Verb', 'Adjective']
    korean_words = [word for word, pos in tokens if pos in selected_pos and word not in korean_stopwords]
    
    # 영어 단어 추출
    english_words = re.findall(r'\b[a-z]+\b', text)
    english_words = [word for word in english_words if word not in english_stopwords]
    
    # 최종 단어 리스트 결합
    words = korean_words + english_words
    return ' '.join(words)


def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            token_type_ids=inputs.get('token_type_ids', None),
            output_hidden_states=True,
            return_dict=True
        )
    # [CLS] 토큰의 임베딩 추출
    embedding = outputs['hidden_states'][-1][:, 0, :].detach().numpy()
    return embedding.squeeze()



# 가중치 설정
goals_area_weight = 0.5         # 가장 높은 가중치
experience_area_weight = 0.3    # 중간 가중치
goals_skills_weight = 0.2       # 중간보다 낮은 가중치


# 적합도 계산 함수
def calculate_fit(mentee, mentors):
    fit_scores = []

    # 멘티의 goals와 experience 전처리 및 임베딩 생성
    mentee_goals_processed = preprocess_text(mentee["goals"])
    mentee_goals_embedding = get_embedding(mentee_goals_processed)

    mentee_experience_processed = preprocess_text(mentee["experience"])
    mentee_experience_embedding = get_embedding(mentee_experience_processed)

    for mentor in mentors:
        # 멘토의 mentoring_area와 skills 전처리 및 임베딩 생성
        mentor_area_processed = preprocess_text(' '.join(mentor["mentoring_area"]))
        mentor_area_embedding = get_embedding(mentor_area_processed)

        mentor_skills_processed = preprocess_text(' '.join(mentor["skills"]))
        mentor_skills_embedding = get_embedding(mentor_skills_processed)

        # 유사도 계산
        # 1. 멘티의 goals와 멘토의 mentoring_area 간의 유사도
        goals_area_similarity = cosine_similarity(
            [mentee_goals_embedding],
            [mentor_area_embedding]
        )[0][0]

        # 2. 멘티의 experience와 멘토의 mentoring_area 간의 유사도
        experience_area_similarity = cosine_similarity(
            [mentee_experience_embedding],
            [mentor_area_embedding]
        )[0][0]

        # 3. 멘티의 goals와 멘토의 skills 간의 유사도
        goals_skills_similarity = cosine_similarity(
            [mentee_goals_embedding],
            [mentor_skills_embedding]
        )[0][0]

        # 가중치 적용하여 총 유사도 계산
        total_similarity = (
            goals_area_similarity * goals_area_weight +
            experience_area_similarity * experience_area_weight +
            goals_skills_similarity * goals_skills_weight
        )

        # 레벨 일치 여부에 따른 작은 가중치 적용
        if mentee["level"] == mentor["desired_mentee_level"]:
            level_bonus = 0.05  # 작은 보너스
        else:
            level_bonus = 0.05

        # 총합 점수 계산
        total_score = total_similarity + level_bonus
        fit_scores.append((mentor["name"], total_score))

    return sorted(fit_scores, key=lambda x: x[1], reverse=True)

# 멘토와의 적합도 계산 결과 출력
fit_results = calculate_fit(mentee, mentors)
print("멘토와의 적합도 점수:")
for mentor_name, score in fit_results:
    print(f"{mentor_name}: {score:.4f}")


Some weights of CustomElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\smhrd1\AppData\Local\Temp\ipykernel_2044\3861715433.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowliste

멘토와의 적합도 점수:
류진영: 0.9917
박지훈: 0.9914
정민수: 0.9875
이서연: 0.9845
김나영: 0.9757
오승민: 0.9695
한재민: 0.9676
최하영: 0.9392
이하늘: 0.9259
김현우: 0.9001


In [11]:
fit_results

[('김영훈', np.float32(0.980444)),
 ('이준혁', np.float32(0.97324747)),
 ('정예은', np.float32(0.9721156)),
 ('이현수', np.float32(0.9591866)),
 ('류시현', np.float32(0.92420393)),
 ('최성호', np.float32(0.92271453)),
 ('오지훈', np.float32(0.8229984)),
 ('박서현', np.float32(0.7999165)),
 ('한미정', np.float32(0.79189366)),
 ('김하윤', np.float32(0.60979855))]